In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip -q install transformers
import numpy as np

In [5]:
!git clone https://github.com/amnghd/Persian_poems_corpus.git

Cloning into 'Persian_poems_corpus'...
remote: Enumerating objects: 159, done.
remote: Total 159 (delta 0), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (159/159), 45.21 MiB | 25.00 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [6]:
!git clone https://github.com/khashei/bolbolzaban-gpt2-persian.git

Cloning into 'bolbolzaban-gpt2-persian'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 31 (delta 9), reused 15 (delta 4), pack-reused 0
Unpacking objects: 100% (31/31), 2.17 MiB | 7.34 MiB/s, done.


In [7]:
from pathlib import Path

def read_poetry(file_path):
    """
    Reads the content of a poetry file.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    formatted_lines = [f"[BOM] {line}" for line in lines if line.strip()]

    return "\n".join(formatted_lines)


def process_poetry_files(folder_path, output_file_path, additional_train_path, additional_test_path):
    """
    Reads and formats all poetry files in the folder, then saves the formatted content to the output file.
    Appends additional train and test data to the output file.
    """
    folder_path = Path(folder_path)
    files = [f for f in folder_path.iterdir() if f.is_file()]

    with open(output_file_path, 'w', encoding='utf-8') as train_file:
        for file_path in files:
            formatted_poetry = read_poetry(file_path)
            train_file.write(formatted_poetry)
            print(f"Formatted content from {file_path.name} added to {output_file_path}")

        for additional_path in [additional_train_path, additional_test_path]:
            additional_path = Path(additional_path)
            if additional_path.exists():
                with open(additional_path, 'r', encoding='utf-8') as additional_file:
                    additional_content = additional_file.read()
                    train_file.write(additional_content )
                    print(f"Content from {additional_path.name} added to {output_file_path}")

    print(f"All formatted content saved to {output_file_path}")


# Usage example
folder_path = '/kaggle/working/Persian_poems_corpus/stop words removed'
train_file_path = '/kaggle/working/train.txt'
additional_train_path = '/kaggle/working/bolbolzaban-gpt2-persian/data/train.txt'
additional_test_path = '/kaggle/working/bolbolzaban-gpt2-persian/data/test.txt'

process_poetry_files(folder_path, train_file_path, additional_train_path, additional_test_path)



Formatted content from onsori_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from abusaeed_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from manoochehri_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from seyf_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from saadi_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from amir_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from vahshi_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from shabestari_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from saeb_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from farrokhi_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from moulavi_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from eraghi_stp_rmv.txt added to /kaggle/working/train.txt
Formatted content from bahaee_stp_rmv.txt added to /kaggle/working/tr

In [8]:
with open('/kaggle/working/train.txt') as f:
    lines = f.readlines()

formatted_lines = [line for line in lines if line.strip()]

new_lines = []
for i in range(0, len(formatted_lines), 2):
    if i + 1 < len(formatted_lines):
        if formatted_lines[i + 1].endswith('\n'):
            new_lines.append(formatted_lines[i] + ' ' + formatted_lines[i + 1].rstrip('\n') + ' [EOS]\n')
        else:
            new_lines.append(formatted_lines[i] + ' ' + formatted_lines[i + 1] + ' [EOS]\n')

In [9]:
new_lines[:30]

['[BOM] دل کار هوا\n [BOM] مشکبوی سلب مشکبوی صبا [EOS]\n',
 '[BOM] رنگ بوی دانم ندانم\n [BOM] هوا صبا صبا هوا [EOS]\n',
 '[BOM] درخت علم پرنیان گشاد\n [BOM] خاک کشیدست مفرش دیبا [EOS]\n',
 '[BOM] بنور ظلمت زمین ابر\n [BOM] بدر مینا سرشک ابر گیا [EOS]\n',
 '[BOM] فریفته زمین ابر تیره ازو\n [BOM] ستاند مینا [EOS]\n',
 '[BOM] بزیر گوهر الوان نقش بدیع\n [BOM] نهفته ازای عالم پهنا [EOS]\n',
 '[BOM] گوهر نقش جهان فراوانست\n [BOM] صناعت ابرست دست صبا [EOS]\n',
 '[BOM] فایده نقش بهار پیکر\n [BOM] هواش جمالست بخار نوا [EOS]\n',
 '[BOM] هواش روزگار\n [BOM] بروزگار خزان هوا کندش هبا [EOS]\n',
 '[BOM] بهار نعت خداوند خسرو عجم\n [BOM] بوستان ازو طبع خاطر شعرا [EOS]\n',
 '[BOM] بهار معنی رنگ بهار حکمت بوی\n [BOM] بهار عقل ثبات بهار کوه بقا [EOS]\n',
 '[BOM] صفت جایگاه مرتبت\n [BOM] مدیح شاه جهان شهریار همتا [EOS]\n',
 '[BOM] یمین دولت مجد امین ملت صدق\n [BOM] امیر غازی محمود سیدالامرا [EOS]\n',
 '[BOM] آفتاب جهان مردمیش\n [BOM] احوال خلا ملا [EOS]\n',
 '[BOM] پدید شب مردمیش\n [BOM] بشب آفتاب نا [EOS

In [10]:
output_file_path = '/kaggle/working/train.txt'

with open(output_file_path, 'w', encoding='utf-8') as f:
    for line in new_lines:
      splited_line = line.split('\n')

      f.writelines(splited_line[0] + splited_line[1] + '\n')


print(f"Formatted lines have been written to {output_file_path}")

Formatted lines have been written to /kaggle/working/train.txt


In [11]:
with open('/kaggle/working/train.txt') as f:
    chr_data = f.read()
    f.seek(0)
    lines = f.readlines()

print('length of characters: ', len(chr_data))
print('length of line: ', len(lines))
print(lines[:10])

length of characters:  37606258
length of line:  643258
['[BOM] دل کار هوا [BOM] مشکبوی سلب مشکبوی صبا [EOS]\n', '[BOM] رنگ بوی دانم ندانم [BOM] هوا صبا صبا هوا [EOS]\n', '[BOM] درخت علم پرنیان گشاد [BOM] خاک کشیدست مفرش دیبا [EOS]\n', '[BOM] بنور ظلمت زمین ابر [BOM] بدر مینا سرشک ابر گیا [EOS]\n', '[BOM] فریفته زمین ابر تیره ازو [BOM] ستاند مینا [EOS]\n', '[BOM] بزیر گوهر الوان نقش بدیع [BOM] نهفته ازای عالم پهنا [EOS]\n', '[BOM] گوهر نقش جهان فراوانست [BOM] صناعت ابرست دست صبا [EOS]\n', '[BOM] فایده نقش بهار پیکر [BOM] هواش جمالست بخار نوا [EOS]\n', '[BOM] هواش روزگار [BOM] بروزگار خزان هوا کندش هبا [EOS]\n', '[BOM] بهار نعت خداوند خسرو عجم [BOM] بوستان ازو طبع خاطر شعرا [EOS]\n']


In [12]:
def split_data(data, train_ratio=0.8):
    """
    Splits the data into train and test sets based on the given ratio.
    """
    split_idx = int(len(data) * train_ratio)
    return data[:split_idx], data[split_idx:]


# Split the data into train and test sets
train, test = split_data(lines)
print(train[:20])
print(test[:20])

['[BOM] دل کار هوا [BOM] مشکبوی سلب مشکبوی صبا [EOS]\n', '[BOM] رنگ بوی دانم ندانم [BOM] هوا صبا صبا هوا [EOS]\n', '[BOM] درخت علم پرنیان گشاد [BOM] خاک کشیدست مفرش دیبا [EOS]\n', '[BOM] بنور ظلمت زمین ابر [BOM] بدر مینا سرشک ابر گیا [EOS]\n', '[BOM] فریفته زمین ابر تیره ازو [BOM] ستاند مینا [EOS]\n', '[BOM] بزیر گوهر الوان نقش بدیع [BOM] نهفته ازای عالم پهنا [EOS]\n', '[BOM] گوهر نقش جهان فراوانست [BOM] صناعت ابرست دست صبا [EOS]\n', '[BOM] فایده نقش بهار پیکر [BOM] هواش جمالست بخار نوا [EOS]\n', '[BOM] هواش روزگار [BOM] بروزگار خزان هوا کندش هبا [EOS]\n', '[BOM] بهار نعت خداوند خسرو عجم [BOM] بوستان ازو طبع خاطر شعرا [EOS]\n', '[BOM] بهار معنی رنگ بهار حکمت بوی [BOM] بهار عقل ثبات بهار کوه بقا [EOS]\n', '[BOM] صفت جایگاه مرتبت [BOM] مدیح شاه جهان شهریار همتا [EOS]\n', '[BOM] یمین دولت مجد امین ملت صدق [BOM] امیر غازی محمود سیدالامرا [EOS]\n', '[BOM] آفتاب جهان مردمیش [BOM] احوال خلا ملا [EOS]\n', '[BOM] پدید شب مردمیش [BOM] بشب آفتاب نا [EOS]\n', '[BOM] وقتش پیشه کار [BOM] ندید نبیندش

In [13]:
# Save the train and test sets to text files
def save_to_txt(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(item + '\n')

save_to_txt(train, '/kaggle/working/train.txt')
save_to_txt(test, '/kaggle/working/tese.txt')

In [15]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel

#model_name = "distilgpt2"
#tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#model_name = "openai-community/gpt2"
model_name = "HooshvareLab/gpt2-fa-poetry"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Define a function to tokenize data line by line
def tokenize_line_by_line(data_path, tokenizer, max_length = 1024):
    tokenized_data = []
    with open(data_path) as f:
      for line in f:
          tokens = tokenizer(line, return_tensors="pt", truncation=True, max_length=max_length)
          # Handle the case where the tokenized input is a single token
          if tokens['input_ids'].squeeze().dim() == 0:  # Check if it's a scalar (0-dimensional tensor)
              tokenized_data.append(tokens['input_ids'].squeeze().item())  # Append the single integer
          else:
              tokenized_data.extend(tokens['input_ids'].squeeze().tolist())  # Extend with the list of tokens

    vocab_size = tokenizer.vocab_size
    print('vocab size of train data is: ',vocab_size)

    return tokenized_data

def decode(token_ids, tokenizer):
    return tokenizer.decode(token_ids, skip_special_tokens=True)


train_file_path = "/kaggle/working/train.txt"
test_file_path = "/kaggle/working/tese.txt"

train_dataset = tokenize_line_by_line(train_file_path, tokenizer)
test_dataset = tokenize_line_by_line(test_file_path, tokenizer)

print("Train dataset loaded successfully!")
print("Test dataset loaded successfully!")


vocab size of train data is:  42000
vocab size of train data is:  42000
Train dataset loaded successfully!
Test dataset loaded successfully!


In [16]:
vocab_size = tokenizer.vocab_size
print('vocab size of train data is: ',vocab_size)

vocab size of train data is:  42000


In [17]:
with open(train_file_path, 'r', encoding='utf-8') as file:
    text = file.read()
tokens = tokenizer.tokenize(text)[:256]
token_ids = tokenizer.convert_tokens_to_ids(tokens)
decoded_text = tokenizer.decode(token_ids)

print("First 256 tokens:")
print(decoded_text)

2024-07-27 10:08:59.474979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 10:08:59.475097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 10:08:59.613628: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


First 256 tokens:
[BOM] دل کار هوا [BOM] مشکبوی سلب مشکبوی صبا [EOS]

[BOM] رنگ بوی دانم ندانم [BOM] هوا صبا صبا هوا [EOS]

[BOM] درخت علم پرنیان گشاد [BOM] خاک کشیدست مفرش دیبا [EOS]

[BOM] بنور ظلمت زمین ابر [BOM] بدر مینا سرشک ابر گیا [EOS]

[BOM] فریفته زمین ابر تیره ازو [BOM] ستاند مینا [EOS]

[BOM] بزیر گوهر الوان نقش بدیع [BOM] نهفته ازای عالم پهنا [EOS]

[BOM] گوهر نقش جهان فراوانست [BOM] صناعت ابرست دست صبا [EOS]

[BOM] فایده نقش بهار پیکر [BOM] هواش جمالست بخار نوا [EOS]

[BOM] هواش روزگار [BOM] بروزگار خزان هوا کندش هبا [EOS]

[BOM] بهار نعت خداوند خسرو عجم [BOM] بوستان ازو طبع خاطر شعرا [EOS]

[BOM] بهار معنی رنگ


In [18]:
import os

os.makedirs('/kaggle/working/Token', exist_ok=True)
print(os.listdir('/kaggle/working'))

['train.txt', '.virtual_documents', 'bolbolzaban-gpt2-persian', 'Persian_poems_corpus', 'tese.txt', 'Token']


In [19]:
train_tokens = np.array(train_dataset)
np.save('/kaggle/working/Token/poem_token_train.npy', train_tokens)

In [20]:
with open(test_file_path, 'r', encoding='utf-8') as file:
    text = file.read()
tokens = tokenizer.tokenize(text)[:256]
token_ids = tokenizer.convert_tokens_to_ids(tokens)
decoded_text = tokenizer.decode(token_ids)

print("First 256 tokens:")
print(decoded_text)

First 256 tokens:
[BOM] مرغ عالم پاک [BOM] دوستان خروش گریه میغ [EOS]

[BOM] برفت جهان عیینه [BOM] گوش ریا خروش شنید [EOS]

[BOM] موکنان عیینه دوید [BOM] نقش زمین نگاری [EOS]

[BOM] غرق خون نازنین شکاری [BOM] چشمه سار سینه تنگ [EOS]

[BOM] خلعت سروش ارغوانی رنگ [BOM] دست سیمین خضاب خون [EOS]

[BOM] چهره گلگونه جامه گلگون [BOM] چهره خون خاک مالید [EOS]

[BOM] دل دردناک نالید [BOM] کای عیینه افتاد [EOS]

[BOM] کآفتاب زوال افتاد [BOM] سیرم عمر لقای [EOS]

[BOM] بودمی [BOM] عقل عشق زند خنده [EOS]

[BOM] بمیری زار زنده [BOM] بگفت جان برآورد آه [EOS]

[BOM] آه جان همراه [BOM] زندگی وفا نشمرد [EOS]

[BOM] نهاد بمرد [BOM


In [21]:
val_tokens = np.array(test_dataset)
np.save('/kaggle/working/Token/poem_token_test.npy', val_tokens)

In [27]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel
model_name = "HooshvareLab/gpt2-fa-poetry"
model = GPT2LMHeadModel.from_pretrained(model_name)

inputs = tokenizer("سلام عزیز بی نظیر", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

In [28]:
# calculate the number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 118099200


# **NanoGPT**

In [ ]:
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
import os
import math
import time
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
#from hellaswag import render_example, iterate_examples
# -----------------------------------------------------------------------------

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True) # flash attention
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 42000 # 50257 number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12 # number of layers
    n_head: int = 12 # number of heads
    n_embd: int = 768 # embedding dimension

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing scheme
        self.transformer.wte.weight = self.lm_head.weight

        # init params
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None, past_key_values=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        # forward the token and posisition embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
        x = tok_emb + pos_emb
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        #################################################
        #model_hf = GPT2LMHeadModel.from_pretrained(model_type)

        model_hf = GPT2LMHeadModel.from_pretrained("openai-community/gpt2") #124 میلیون پارامتر

        #model_name = "distilgpt2" #82 میلیون پارامتر
        #model_hf = GPT2LMHeadModel.from_pretrained(model_name)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def configure_optimizers(self, weight_decay, learning_rate, device_type):
        # start with all of the candidate parameters (that require grad)
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        if master_process:
            print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
            print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        if master_process:
            print(f"using fused AdamW: {use_fused}")
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

# -----------------------------------------------------------------------------
#import tiktoken
import numpy as np

def load_tokens(filename):
    npt = np.load(filename, allow_pickle=True)
    npt = npt.astype(np.int32) # added after video
    ptt = torch.tensor(npt, dtype=torch.long)
    return ptt



class DataLoaderLite:
    def __init__(self, B, T, process_rank, num_processes, split):
        self.B = B
        self.T = T
        self.process_rank = process_rank
        self.num_processes = num_processes
        assert split in {'train', 'test'}

        # get the shard filenames
        data_root = "/kaggle/working/Token"
        shards = os.listdir(data_root)
        shards = [s for s in shards if split in s]
        shards = sorted(shards)
        shards = [os.path.join(data_root, s) for s in shards]
        self.shards = shards
        assert len(shards) > 0, f"no shards found for split {split}"
        if master_process:
            print(f"found {len(shards)} shards for split {split}")
        self.reset()

    def reset(self):
        # state, init at shard zero
        self.current_shard = 0
        self.tokens = load_tokens(self.shards[self.current_shard])
        self.current_position = self.B * self.T * self.process_rank

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # targets
        # advance the position in the tensor
        self.current_position += B * T * self.num_processes
        # if loading the next batch would be out of bounds, advance to next shard
        if self.current_position + (B * T * self.num_processes + 1) > len(self.tokens):
            self.current_shard = (self.current_shard + 1) % len(self.shards)
            self.tokens = load_tokens(self.shards[self.current_shard])
            self.current_position = B * T * self.process_rank
        return x, y


# -----------------------------------------------------------------------------
# helper function for HellaSwag eval
# takes tokens, mask, and logits, returns the index of the completion with the lowest loss

def get_most_likely_row(tokens, mask, logits):
    # evaluate the autoregressive loss at all positions
    shift_logits = (logits[..., :-1, :]).contiguous()
    shift_tokens = (tokens[..., 1:]).contiguous()
    flat_shift_logits = shift_logits.view(-1, shift_logits.size(-1))
    flat_shift_tokens = shift_tokens.view(-1)
    shift_losses = F.cross_entropy(flat_shift_logits, flat_shift_tokens, reduction='none')
    shift_losses = shift_losses.view(tokens.size(0), -1)
    # now get the average loss just for the completion region (where mask == 1), in each row
    shift_mask = (mask[..., 1:]).contiguous() # we must shift mask, so we start at the last prompt token
    masked_shift_losses = shift_losses * shift_mask
    # sum and divide by the number of 1s in the mask
    sum_loss = masked_shift_losses.sum(dim=1)
    avg_loss = sum_loss / shift_mask.sum(dim=1)
    # now we have a loss for each of the 4 completions
    # the one with the lowest loss should be the most likely
    pred_norm = avg_loss.argmin().item()
    return pred_norm

# -----------------------------------------------------------------------------
# simple launch:
# python train_gpt2.py
# DDP launch for e.g. 8 GPUs:
# torchrun --standalone --nproc_per_node=8 train_gpt2.py

# run the training loop
from torch.distributed import init_process_group, destroy_process_group
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

# set up DDP (distributed data parallel).
# torchrun command sets the env variables RANK, LOCAL_RANK, and WORLD_SIZE
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    # use of DDP atm demands CUDA, we set the device appropriately according to rank
    assert torch.cuda.is_available(), "for now i think we need CUDA for DDP"
    init_process_group(backend='nccl')
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
else:
    # vanilla, non-DDP run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True
    # attempt to autodetect device
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = "mps"
    print(f"using device: {device}")

# added after video, pytorch can be serious about it's device vs. device_type distinction
device_type = "cuda" if device.startswith("cuda") else "cpu"

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)


total_batch_size = 2048  # 2**19, ~0.5M, in number of tokens
B = 4 # micro batch size
T = 512 # sequence length
assert total_batch_size % (B * T * ddp_world_size) == 0, "make sure total_batch_size is divisible by B * T * ddp_world_size"
grad_accum_steps = total_batch_size // (B * T * ddp_world_size)
if master_process:
    print(f"total desired batch size: {total_batch_size}")
    print(f"=> calculated gradient accumulation steps: {grad_accum_steps}")

train_loader = DataLoaderLite(B=B, T=T, process_rank=ddp_rank, num_processes=ddp_world_size, split="train")
val_loader = DataLoaderLite(B=B, T=T, process_rank=ddp_rank, num_processes=ddp_world_size, split="test")

torch.set_float32_matmul_precision('high')

# create model
#model = GPT(GPTConfig(vocab_size=50304))
# model = GPT.from_pretrained("gpt2") # or init from OpenAI GPT-2
model.to(device)
use_compile = False # torch.compile interferes with HellaSwag eval and Generation. TODO fix
if use_compile:
    model = torch.compile(model)
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])
raw_model = model.module if ddp else model # always contains the "raw" unwrapped model

max_lr = 6e-5
min_lr = max_lr * 0.1
warmup_steps = 715
max_steps = 1000 # 19,073 steps is ~1 epoch, if data is 10B tokens and batch size 0.5M tokens
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_steps:
        return max_lr * (it+1) / warmup_steps
    # 2) if it > lr_decay_iters, return min learning rate
    if it > max_steps:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff starts at 1 and goes to 0
    return min_lr + coeff * (max_lr - min_lr)

# optimize!
#optimizer = raw_model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device_type=device_type)
optimizer = torch.optim.AdamW(model.parameters(), lr=6e-4, weight_decay=0.1)

# create the log directory we will write checkpoints to and log to
log_dir = "log"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f"log.txt")
with open(log_file, "w") as f: # open for writing to clear the file
    pass

for step in range(max_steps):
    t0 = time.time()
    last_step = (step == max_steps - 1)

    # once in a while evaluate our validation loss
    if step % 250 == 0 or last_step:
        model.eval()
        val_loader.reset()
        with torch.no_grad():
            val_loss_accum = 0.0
            val_loss_steps = 20
            for _ in range(val_loss_steps):
                x, y = val_loader.next_batch()
                x, y = x.to(device), y.to(device)
                with torch.autocast(device_type=device_type, dtype=torch.float16):
                    #logits, loss = model(x, y)
                    outputs = model(x, labels=y)  # Provide labels for loss calculation
                    logits = outputs.logits
                    loss = outputs.loss

                loss = loss / val_loss_steps
                val_loss_accum += loss.detach()
        if ddp:
            dist.all_reduce(val_loss_accum, op=dist.ReduceOp.AVG)
        if master_process:
            print(f"validation loss: {val_loss_accum.item():.4f}")
            with open(log_file, "a") as f:
                f.write(f"{step} val {val_loss_accum.item():.4f}\n")
            if step > 0 and (step % 5000 == 0 or last_step):
                # optionally write model checkpoints
                checkpoint_path = os.path.join(log_dir, f"model_{step:05d}.pt")
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'config': raw_model.config,
                    'step': step,
                    'val_loss': val_loss_accum.item()
                }
                # you might also want to add optimizer.state_dict() and
                # rng seeds etc., if you wanted to more exactly resume training
                torch.save(checkpoint, checkpoint_path)

################################################################################

    # once in a while generate from the model (except step 0, which is noise)
    if ((step > 0) or last_step) and (not use_compile):
        model.eval()
        num_return_sequences = 4
        max_length = 32
        tokens = tokenizer.encode("بهار دلکش")
        tokens = torch.tensor(tokens, dtype=torch.long)
        tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
        xgen = tokens.to(device)
        sample_rng = torch.Generator(device=device)
        sample_rng.manual_seed(42 + ddp_rank)
        while xgen.size(1) < max_length:
            # forward the model to get the logits
            with torch.no_grad():
                with torch.autocast(device_type=device_type, dtype=torch.float16):
                    #logits, loss = model(xgen) # (B, T, vocab_size)
                    outputs = model(xgen)  # Get the model outputs
                    logits = outputs.logits  # Extract the logits
                # take the logits at the last position
                logits = logits[:, -1, :] # (B, vocab_size)
                # get the probabilities
                probs = F.softmax(logits, dim=-1)
                # do top-k sampling of 50 (huggingface pipeline default)
                # topk_probs here becomes (5, 50), topk_indices is (5, 50)
                topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
                # select a token from the top-k probabilities
                # note: multinomial does not demand the input to sum to 1
                ix = torch.multinomial(topk_probs, 1, generator=sample_rng) # (B, 1)
                # gather the corresponding indices
                xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
                # append to the sequence
                xgen = torch.cat((xgen, xcol), dim=1)
        # print the generated text
        for i in range(num_return_sequences):
            tokens = xgen[i, :max_length].tolist()
            decoded = tokenizer.decode(tokens)
            ################################################################################
            #decoded = decoded.replace('[BOM]', ' ').replace('[EOS]', '.').replace('[CLS]', ' ').replace('[SEP]', '"')
            #decoded = ' '.join([part.strip() for part in decoded.split('-')])

            print(f"rank {ddp_rank} sample {i}: {decoded}")

    # do one step of the optimization
    model.train()
    optimizer.zero_grad()
    loss_accum = 0.0
    for micro_step in range(grad_accum_steps):
        x, y = train_loader.next_batch()
        x, y = x.to(device), y.to(device)
        # added after video, this field is also used by the forward pass.
        if ddp:
            model.require_backward_grad_sync = (micro_step == grad_accum_steps - 1)
        with torch.autocast(device_type=device_type, dtype=torch.float16):
            #logits, loss = model(x, y)
            outputs = model(x, labels=y)  # Provide labels for loss calculation
            logits = outputs.logits
            loss = outputs.loss
        # we have to scale the loss to account for gradient accumulation,
        # because the gradients just add on each successive backward().
        # addition of gradients corresponds to a SUM in the objective, but
        # instead of a SUM we want MEAN. Scale the loss here so it comes out right
        loss = loss / grad_accum_steps
        loss_accum += loss.detach()
        loss.backward()
    if ddp:
        dist.all_reduce(loss_accum, op=dist.ReduceOp.AVG)
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    # determine and set the learning rate for this iteration
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    optimizer.step()
    if device_type == "cuda":
        torch.cuda.synchronize() # wait for the GPU to finish work
    t1 = time.time()
    dt = t1 - t0 # time difference in seconds
    tokens_processed = train_loader.B * train_loader.T * grad_accum_steps * ddp_world_size
    tokens_per_sec = tokens_processed / dt
    if master_process:
        print(f"step {step:5d} | loss: {loss_accum.item():.6f} | lr {lr:.4e} | norm: {norm:.4f} | dt: {dt*1000:.2f}ms | tok/sec: {tokens_per_sec:.2f}")
        with open(log_file, "a") as f:
            f.write(f"{step} train {loss_accum.item():.6f}\n")

if ddp:
    destroy_process_group()

using device: cuda
total desired batch size: 2048
=> calculated gradient accumulation steps: 1
found 1 shards for split train
found 1 shards for split test
validation loss: 3.3948
step     0 | loss: 3.786225 | lr 8.3916e-08 | norm: 1.9209 | dt: 2202.90ms | tok/sec: 929.68
rank 0 sample 0: بهار دلکش [OM عشق [OM آتش سوز [OS
[OM عشق سوزی دلبند]
[OM حسن دلبند دلبند دلبندند دلبندند [OM عالم [
rank 0 sample 1: بهار دلکش روزگاری<|startoftext|>بوستانا کار ما شهریست پر خون باغ جان<sep>شبنمی چون گلستان باغ پر لاله باغ جان<sep>جگر خون چمن خون
rank 0 sample 2: بهار دلکش ترانه [OS
[OM شادی م [OM نوش نوشان جان [OM عشق دل [OS
[OM باغ بهاری [OM حسن [OM
rank 0 sample 3: بهار دلکش نوایی انگیز [OM بلبل نوا [OM طوطی نی [OS
[OM شکر تنگ [OM غم جان [OM یار [OM عشق [OS

step     1 | loss: 3.745968 | lr 1.6783e-07 | norm: 1.7933 | dt: 631.64ms | tok/sec: 3242.36
rank 0 sample 0: بهار دلکش [OM عشق [OM آتش سوز [OS
[OM عشق سوزی دلبند]
[OM حسن دلبند دلبند دلبندند دلبندند [OM عالم [
rank 0 sample 1: بهار دلکش روزگا

In [ ]:
import os
import math
import time
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
#from hellaswag import render_example, iterate_examples
# -----------------------------------------------------------------------------

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True) # flash attention
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 42000 # 50257 number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12 # number of layers
    n_head: int = 12 # number of heads
    n_embd: int = 768 # embedding dimension

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing scheme
        self.transformer.wte.weight = self.lm_head.weight

        # init params
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None, past_key_values=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        # forward the token and posisition embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
        x = tok_emb + pos_emb
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        #################################################
        #model_hf = GPT2LMHeadModel.from_pretrained(model_type)

        model_hf = GPT2LMHeadModel.from_pretrained("openai-community/gpt2") #124 میلیون پارامتر

        #model_name = "distilgpt2" #82 میلیون پارامتر
        #model_hf = GPT2LMHeadModel.from_pretrained(model_name)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def configure_optimizers(self, weight_decay, learning_rate, device_type):
        # start with all of the candidate parameters (that require grad)
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        if master_process:
            print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
            print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        if master_process:
            print(f"using fused AdamW: {use_fused}")
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
        return optimizer

# -----------------------------------------------------------------------------
#import tiktoken
import numpy as np

def load_tokens(filename):
    npt = np.load(filename, allow_pickle=True)
    npt = npt.astype(np.int32) # added after video
    ptt = torch.tensor(npt, dtype=torch.long)
    return ptt



class DataLoaderLite:
    def __init__(self, B, T, process_rank, num_processes, split):
        self.B = B
        self.T = T
        self.process_rank = process_rank
        self.num_processes = num_processes
        assert split in {'train', 'test'}

        # get the shard filenames
        data_root = "/kaggle/working/Token"
        shards = os.listdir(data_root)
        shards = [s for s in shards if split in s]
        shards = sorted(shards)
        shards = [os.path.join(data_root, s) for s in shards]
        self.shards = shards
        assert len(shards) > 0, f"no shards found for split {split}"
        if master_process:
            print(f"found {len(shards)} shards for split {split}")
        self.reset()

    def reset(self):
        # state, init at shard zero
        self.current_shard = 0
        self.tokens = load_tokens(self.shards[self.current_shard])
        self.current_position = self.B * self.T * self.process_rank

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # targets
        # advance the position in the tensor
        self.current_position += B * T * self.num_processes
        # if loading the next batch would be out of bounds, advance to next shard
        if self.current_position + (B * T * self.num_processes + 1) > len(self.tokens):
            self.current_shard = (self.current_shard + 1) % len(self.shards)
            self.tokens = load_tokens(self.shards[self.current_shard])
            self.current_position = B * T * self.process_rank
        return x, y


# -----------------------------------------------------------------------------
# helper function for HellaSwag eval
# takes tokens, mask, and logits, returns the index of the completion with the lowest loss

def get_most_likely_row(tokens, mask, logits):
    # evaluate the autoregressive loss at all positions
    shift_logits = (logits[..., :-1, :]).contiguous()
    shift_tokens = (tokens[..., 1:]).contiguous()
    flat_shift_logits = shift_logits.view(-1, shift_logits.size(-1))
    flat_shift_tokens = shift_tokens.view(-1)
    shift_losses = F.cross_entropy(flat_shift_logits, flat_shift_tokens, reduction='none')
    shift_losses = shift_losses.view(tokens.size(0), -1)
    # now get the average loss just for the completion region (where mask == 1), in each row
    shift_mask = (mask[..., 1:]).contiguous() # we must shift mask, so we start at the last prompt token
    masked_shift_losses = shift_losses * shift_mask
    # sum and divide by the number of 1s in the mask
    sum_loss = masked_shift_losses.sum(dim=1)
    avg_loss = sum_loss / shift_mask.sum(dim=1)
    # now we have a loss for each of the 4 completions
    # the one with the lowest loss should be the most likely
    pred_norm = avg_loss.argmin().item()
    return pred_norm

# -----------------------------------------------------------------------------
# simple launch:
# python train_gpt2.py
# DDP launch for e.g. 8 GPUs:
# torchrun --standalone --nproc_per_node=8 train_gpt2.py

# run the training loop
from torch.distributed import init_process_group, destroy_process_group
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

# set up DDP (distributed data parallel).
# torchrun command sets the env variables RANK, LOCAL_RANK, and WORLD_SIZE
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    # use of DDP atm demands CUDA, we set the device appropriately according to rank
    assert torch.cuda.is_available(), "for now i think we need CUDA for DDP"
    init_process_group(backend='nccl')
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
else:
    # vanilla, non-DDP run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True
    # attempt to autodetect device
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = "mps"
    print(f"using device: {device}")

# added after video, pytorch can be serious about it's device vs. device_type distinction
device_type = "cuda" if device.startswith("cuda") else "cpu"

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)


total_batch_size = 2048  # 2**19, ~0.5M, in number of tokens
B = 4 # micro batch size
T = 512 # sequence length
assert total_batch_size % (B * T * ddp_world_size) == 0, "make sure total_batch_size is divisible by B * T * ddp_world_size"
grad_accum_steps = total_batch_size // (B * T * ddp_world_size)
if master_process:
    print(f"total desired batch size: {total_batch_size}")
    print(f"=> calculated gradient accumulation steps: {grad_accum_steps}")

train_loader = DataLoaderLite(B=B, T=T, process_rank=ddp_rank, num_processes=ddp_world_size, split="train")
val_loader = DataLoaderLite(B=B, T=T, process_rank=ddp_rank, num_processes=ddp_world_size, split="test")

torch.set_float32_matmul_precision('high')

# create model
#model = GPT(GPTConfig(vocab_size=50304))
# model = GPT.from_pretrained("gpt2") # or init from OpenAI GPT-2
model.to(device)
use_compile = False # torch.compile interferes with HellaSwag eval and Generation. TODO fix
if use_compile:
    model = torch.compile(model)
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])
raw_model = model.module if ddp else model # always contains the "raw" unwrapped model

max_lr = 6e-5
min_lr = max_lr * 0.1
warmup_steps = 715
max_steps = 18000 # 19,073 steps is ~1 epoch, if data is 10B tokens and batch size 0.5M tokens
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_steps:
        return max_lr * (it+1) / warmup_steps
    # 2) if it > lr_decay_iters, return min learning rate
    if it > max_steps:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff starts at 1 and goes to 0
    return min_lr + coeff * (max_lr - min_lr)

# optimize!
#optimizer = raw_model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device_type=device_type)
optimizer = torch.optim.AdamW(model.parameters(), lr=6e-4, weight_decay=0.1)

# create the log directory we will write checkpoints to and log to
log_dir = "log"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f"log.txt")
with open(log_file, "w") as f: # open for writing to clear the file
    pass

for step in range(max_steps):
    t0 = time.time()
    last_step = (step == max_steps - 1)

    # once in a while evaluate our validation loss
    if step % 250 == 0 or last_step:
        model.eval()
        val_loader.reset()
        with torch.no_grad():
            val_loss_accum = 0.0
            val_loss_steps = 20
            for _ in range(val_loss_steps):
                x, y = val_loader.next_batch()
                x, y = x.to(device), y.to(device)
                with torch.autocast(device_type=device_type, dtype=torch.float16):
                    #logits, loss = model(x, y)
                    outputs = model(x, labels=y)  # Provide labels for loss calculation
                    logits = outputs.logits
                    loss = outputs.loss

                loss = loss / val_loss_steps
                val_loss_accum += loss.detach()
        if ddp:
            dist.all_reduce(val_loss_accum, op=dist.ReduceOp.AVG)
        if master_process:
            print(f"validation loss: {val_loss_accum.item():.4f}")
            with open(log_file, "a") as f:
                f.write(f"{step} val {val_loss_accum.item():.4f}\n")
            if step > 0 and (step % 5000 == 0 or last_step):
                # optionally write model checkpoints
                checkpoint_path = os.path.join(log_dir, f"model_{step:05d}.pt")
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'config': raw_model.config,
                    'step': step,
                    'val_loss': val_loss_accum.item()
                }
                # you might also want to add optimizer.state_dict() and
                # rng seeds etc., if you wanted to more exactly resume training
                torch.save(checkpoint, checkpoint_path)

################################################################################

    # once in a while generate from the model (except step 0, which is noise)
    if ((step > 0) or last_step) and (not use_compile):
        model.eval()
        num_return_sequences = 4
        max_length = 32
        tokens = tokenizer.encode("بهار دلکش")
        tokens = torch.tensor(tokens, dtype=torch.long)
        tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
        xgen = tokens.to(device)
        sample_rng = torch.Generator(device=device)
        sample_rng.manual_seed(42 + ddp_rank)
        while xgen.size(1) < max_length:
            # forward the model to get the logits
            with torch.no_grad():
                with torch.autocast(device_type=device_type, dtype=torch.float16):
                    #logits, loss = model(xgen) # (B, T, vocab_size)
                    outputs = model(xgen)  # Get the model outputs
                    logits = outputs.logits  # Extract the logits
                # take the logits at the last position
                logits = logits[:, -1, :] # (B, vocab_size)
                # get the probabilities
                probs = F.softmax(logits, dim=-1)
                # do top-k sampling of 50 (huggingface pipeline default)
                # topk_probs here becomes (5, 50), topk_indices is (5, 50)
                topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
                # select a token from the top-k probabilities
                # note: multinomial does not demand the input to sum to 1
                ix = torch.multinomial(topk_probs, 1, generator=sample_rng) # (B, 1)
                # gather the corresponding indices
                xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
                # append to the sequence
                xgen = torch.cat((xgen, xcol), dim=1)
        # print the generated text
        for i in range(num_return_sequences):
            tokens = xgen[i, :max_length].tolist()
            decoded = tokenizer.decode(tokens)
            ################################################################################
            #decoded = decoded.replace('[BOM]', ' ').replace('[EOS]', '.').replace('[CLS]', ' ').replace('[SEP]', '"')
            #decoded = ' '.join([part.strip() for part in decoded.split('-')])

            print(f"rank {ddp_rank} sample {i}: {decoded}")

    # do one step of the optimization
    model.train()
    optimizer.zero_grad()
    loss_accum = 0.0
    for micro_step in range(grad_accum_steps):
        x, y = train_loader.next_batch()
        x, y = x.to(device), y.to(device)
        # added after video, this field is also used by the forward pass.
        if ddp:
            model.require_backward_grad_sync = (micro_step == grad_accum_steps - 1)
        with torch.autocast(device_type=device_type, dtype=torch.float16):
            #logits, loss = model(x, y)
            outputs = model(x, labels=y)  # Provide labels for loss calculation
            logits = outputs.logits
            loss = outputs.loss
        # we have to scale the loss to account for gradient accumulation,
        # because the gradients just add on each successive backward().
        # addition of gradients corresponds to a SUM in the objective, but
        # instead of a SUM we want MEAN. Scale the loss here so it comes out right
        loss = loss / grad_accum_steps
        loss_accum += loss.detach()
        loss.backward()
    if ddp:
        dist.all_reduce(loss_accum, op=dist.ReduceOp.AVG)
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    # determine and set the learning rate for this iteration
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    optimizer.step()
    if device_type == "cuda":
        torch.cuda.synchronize() # wait for the GPU to finish work
    t1 = time.time()
    dt = t1 - t0 # time difference in seconds
    tokens_processed = train_loader.B * train_loader.T * grad_accum_steps * ddp_world_size
    tokens_per_sec = tokens_processed / dt
    if master_process:
        print(f"step {step:5d} | loss: {loss_accum.item():.6f} | lr {lr:.4e} | norm: {norm:.4f} | dt: {dt*1000:.2f}ms | tok/sec: {tokens_per_sec:.2f}")
        with open(log_file, "a") as f:
            f.write(f"{step} train {loss_accum.item():.6f}\n")

if ddp:
    destroy_process_group()


using device: cuda
total desired batch size: 2048
=> calculated gradient accumulation steps: 1
found 1 shards for split train
found 1 shards for split test
validation loss: 3.2336
step     0 | loss: 3.608654 | lr 8.3916e-08 | norm: 2.0940 | dt: 2201.49ms | tok/sec: 930.28
rank 0 sample 0: بهار دلکش [OS
B]
[OM خزان حله [OM باغو سبزه باغ [OM مهرگانB]
[OM بوی [OM خانه [OS
rank 0 sample 1: بهار دلکشB] بوسه بوسه بوسه بوسهB] بوسهE] بوسه [OS
[OM لعل بوسه [OM بوسه [OS
[OM لعل زلف بوسه
rank 0 sample 2: بهار دلکشE]
[OM سرو بلندB] نتر [OS
[OM کوهB] [OS
[OM کوه [OM کوه [OM
rank 0 sample 3: بهار دلکشB] نوبه لاله لاله [OM باغ باغ نرگسE]
[OM گل ماه [OM باغ نسرین [OS
[OM ماه [OM گلب
step     1 | loss: 3.535238 | lr 1.6783e-07 | norm: 1.9851 | dt: 628.50ms | tok/sec: 3258.54
rank 0 sample 0: بهار دلکش [OS
B]
[OM خزان حله [OM باغ شکوفه [OS
[OM بنفشهB] بنفشهست [OM لاله [OS

rank 0 sample 1: بهار دلکشB] بوسه بوسه بوسه بوسهB] بوسهE] بوسه [OS
[OM لعل بوسه [OM بوسه [OS
[OM لعل زلف بوسه
rank 0 sample 2: بهار 